### Замерим качество Линейной регрессии после обработки данных не просто на отложенной выборке, но и на Кросс-Валидации на 4 фолдах!

In [2]:
import numpy as np
import pandas as pd

processed_data = pd.read_csv('processed_data.csv', index_col='id')

In [3]:
processed_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration
id,,,,,
id2875421,1,930.399753,0,1.500479,6.122493
id2377394,0,930.399753,0,1.807119,6.498282
id3858529,1,930.399753,0,6.392080,7.661527
id3504673,1,930.399753,0,1.487155,6.063785
id2181028,1,930.399753,0,1.189925,6.077642


#### ! Не перемешивайте данные

In [4]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn import metrics

selector = KFold(n_splits=4)

### Задание 4
### Замерьте качество (MSLE, как и раньше) на Кросс-валидации, 
### используя MSE от log_trip_duration и назначенный selector

X = processed_data.drop('log_trip_duration', axis = 1)
y = processed_data['log_trip_duration']

losses_train = []
losses_test = []

for train_index, test_index in selector.split(X):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    y_train, y_test = y.values[train_index], y.values[test_index]
    
    model = LinearRegression()
    model.fit(X_train, y_train)
      
    losses_train.append(metrics.mean_squared_error(model.predict(X_train), y_train))
    losses_test.append(metrics.mean_squared_error(model.predict(X_test), y_test))

cross_val_error = np.mean(losses_test)
print(f"MSLE на Кросс-валидации: {round(cross_val_error, 3)}")

MSLE на Кросс-валидации: 0.426


## Поработал один из хитрых гномов!

В отличие от своих собратьев, третий гном оказался тем еще бездельником в школьные годы, но все равно страстно желал во всем догнать первых двух. И сейчас, желая помочь им в построении модели по предсказанию длительности поездки такси, добавил в данные 20 зашифрованных фичей (их смысл нам не рассказали: какая-то секретная информация о водителях).

Гном думал следующим образом: "Ну не может же модель стать хуже! А тут вот авось и мое нововведение уменьшит ошибку в разы! Тогда и меня станут звать на гномий  data-саммит."

Проверим на кросс-валидации, насколько гном оказался прав!

In [5]:
new_data = pd.read_csv('new_data.csv', index_col='id')

In [6]:
new_data.head()

,vendor_id,passenger_count,store_and_fwd_flag,distance_km,log_trip_duration,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20
id,,,,,,,,,,,,,,,,,,,,,
id2875421,1,930.399753,0,1.500479,6.122493,1,1,1,1,1,...,0,0,0,0,0,1.500479,2.251437,3.378234,5.068969,7.605881
id2377394,0,930.399753,0,1.807119,6.498282,0,0,0,0,0,...,0,0,0,0,0,1.807119,3.265681,5.901475,10.664670,19.272331
id3858529,1,930.399753,0,6.392080,7.661527,1,1,1,1,1,...,0,0,0,0,0,6.392080,40.858690,261.172025,1669.432545,10671.146803
id3504673,1,930.399753,0,1.487155,6.063785,1,1,1,1,1,...,0,0,0,0,0,1.487155,2.211629,3.289035,4.891303,7.274125
id2181028,1,930.399753,0,1.189925,6.077642,1,1,1,1,1,...,0,0,0,0,0,1.189925,1.415923,1.684842,2.004837,2.385606


In [7]:
### Задание №5
### Замерьте качество (MSLE, как и раньше) на Кросс-валидации, 
### используя MSE от log_trip_duration и назначенный ранее selector

X_new = new_data.drop('log_trip_duration', axis = 1)
y_new = new_data['log_trip_duration']

losses_train_new = []
losses_test_new = []

for train_index, test_index in selector.split(X_new):
    
    X_train_, X_test_ = X_new.values[train_index], X_new.values[test_index]
    y_train_, y_test_ = y_new.values[train_index], y_new.values[test_index]
    
    model_2 = LinearRegression()
    model_2.fit(X_train_, y_train_)
    
    losses_train_new.append(np.mean((model_2.predict(X_train_) - y_train_)**2))
    losses_test_new.append(np.mean((model_2.predict(X_test_) - y_test_)**2))
    
cross_val_error_2 = np.mean(losses_test_new)
    

print(f"MSLE на Кросс-валидации: {round(cross_val_error_2, 3)}")

MSLE на Кросс-валидации: 140.921


В линейной алгербре зачастую используют понятие **ранга матрицы**. Оно соответствует кол-ву линейно независимых столбцов в матрице. Иными словами, позволяет оценить, есть ли избыток информации в нашем датафрейме. Если ранг матрицы меньше, чем кол-во используемых столбцов, то некоторые фичи следует удалить, ведь иначе возникает ситуация строгой мультиколлинеарности.

Чтобы замерить ранг в наших матрицах объект-признак, можно воспользоваться функцией numpy.linalg.matrix_rank

Константным признаком в данном упражнении можно пренебречь.

In [8]:
### Создайте переменные rank_processed, rank_new 
### Соответственно равные рангу изначальной матрицы
### с данными и рангу матрицы третьего гнома

rank_processed = np.linalg.matrix_rank(X)
rank_new = np.linalg.matrix_rank(X_new)

print(f'rank_processed {rank_processed}')
print(f'rank_new {rank_new}')

rank_processed 4
rank_new 5


In [9]:
### Создайте переменные num_features_processed, num_features_new
### Соответственно равные кол-ву фичей в изначальной матрицы
### с данными и кол-ву фичей у третьего гнома

num_features_processed = X.shape[1]
num_features_new = X_new.shape[1]

print(f'num_features_processed {num_features_processed}')
print(f'num_features_new {num_features_new}')



num_features_processed 4
num_features_new 24


In [10]:
### Задание №6
print(f"В первой модели всего фичей: {num_features_processed}, - а ранг равен {rank_processed}")

print(f"Во второй модели всего фичей: {num_features_new}, - а ранг равен {rank_new}")

В первой модели всего фичей: 4, - а ранг равен 4
Во второй модели всего фичей: 24, - а ранг равен 5


Не кажется ли нам, что из-за новых 20 фичей появилась проблема мультиколлинеарности? Как поступить гному, чтобы, с одной стороны, получить адекватное качество, а с другой стороны, не повредить свое самолюбие и не убирать новые признаки?

Верно! Например, с помощью регуляризации.

Найдите такой параметр регуляризации $\alpha$ для Ridge и Lasso случая, чтобы ошибка RMSLE на кросс-валидации оказалась строго меньше 0.4

**ALARM**: используйте процедуру масштабирования данных (воспользуйтесь методом MinMaxScaler) перед тем как применить регуляризацию. Важно - чтобы сохранить концепцию независимости обучения на трейне и на тесте, на каждой итерации кросс-валидации необходимо замерять параметры стандартизации исключительно на трейне, а потом применять на валидационном фолде.

In [11]:
### Пример, как это можно сделать в цикле
### То есть обучить Lasso, учитывая масштабирование
### Исключительно на трейне на каждой итерации



X = new_data.drop('log_trip_duration', axis=1)
Y = new_data['log_trip_duration']

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn import metrics

scores = []

for train_index, test_index in selector.split(X):
    
    X_train, X_test = X.values[train_index], X.values[test_index]
    Y_train, Y_test = Y.values[train_index], Y.values[test_index]
    
    ### Фитим исключительно на трейне!
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    
    ### Применяем обученный scaler на трейн и тест
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    ### max_iter иногда требуется ставить побольше, 
    ### особенно когда данных много и/или они сложные
    ### этот параметр регулирует верхнюю границу кол-ва
    ### итераций во время обучения
    ### подробнее - в документации
    
    ### По дефолту здесь параметр регуляризации alpha=1
    
    model_lasso = Ridge(alpha = 10002) 
    model_lasso.fit(X_train_scaled, Y_train)
    
    predictions = model_lasso.predict(X_test_scaled)
    
    scores.append(metrics.mean_squared_error(predictions, Y_test))

    
print(f"MSLE на Кросс-валидации равен: {np.mean(scores)}")

### P.S. если вы уже умеете работать с конструкциями
### Pipeline, GridSearchCV, cross_validate
### Можете использовать их. Мы познакомимся с ними позже.

MSLE на Кросс-валидации равен: 0.6313156161299629


In [13]:
### Теперь найдите оптимальный параметр регуляризации
### для случая Ridge
### Напомним: ошибка на кросс-валидации должно быть 
### строго меньше 0.4

### Задание №7
\
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV


X_1 = new_data.drop('log_trip_duration', axis=1)
Y_1 = new_data['log_trip_duration']

pipe = Pipeline([('scaler', MinMaxScaler()), ('ridge', Ridge())])

alphas = np.linspace(0.001, 1, num=30)


grid = GridSearchCV(pipe, param_grid= {'ridge__alpha' : alphas}, scoring='neg_mean_squared_error', cv=selector,
                        return_train_score=True)

grid.fit(X_1, Y_1)



GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': array([0.001     , 0.03544828, 0.06989655, 0.10434483, 0.1387931 ,
       0.17324138, 0.20768966, 0.24213793, 0.27658621, 0.31103448,
       0.34548276, 0.37993103, 0.41437931, 0.44882759, 0.48327586,
       0.51772414, 0.55217241, 0.58662069, 0.62106897, 0.65551724,
       0.68996552, 0.72441379, 0.75886207, 0.79331034, 0.82775862,
       0.8622069 , 0.89665517, 0.93110345, 0.96555172, 1.        ])},
             return_train_score=True, scoring='neg_mean_squared_error')

In [16]:
grid.score

<bound method BaseSearchCV.score of GridSearchCV(cv=KFold(n_splits=4, random_state=None, shuffle=False),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('ridge', Ridge())]),
             param_grid={'ridge__alpha': array([0.001     , 0.03544828, 0.06989655, 0.10434483, 0.1387931 ,
       0.17324138, 0.20768966, 0.24213793, 0.27658621, 0.31103448,
       0.34548276, 0.37993103, 0.41437931, 0.44882759, 0.48327586,
       0.51772414, 0.55217241, 0.58662069, 0.62106897, 0.65551724,
       0.68996552, 0.72441379, 0.75886207, 0.79331034, 0.82775862,
       0.8622069 , 0.89665517, 0.93110345, 0.96555172, 1.        ])},
             return_train_score=True, scoring='neg_mean_squared_error')>

In [72]:
### Найдите оптимальный параметр регуляризации
### для случая Lasso
### Напомним: ошибка на кросс-валидации должно быть 
### строго меньше 0.4

### Задание №8
### Your code is here

X_2 = new_data.drop('log_trip_duration', axis=1)
Y_2 = new_data['log_trip_duration']

from sklearn.linear_model import Lasso

scores_2 = []

for train_index, test_index in selector.split(X_2):
    
    X_train, X_test = X_2.values[train_index], X_2.values[test_index]
    Y_train, Y_test = Y_2.values[train_index], Y_2.values[test_index]
    
    ### Фитим исключительно на трейне!
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    
    ### Применяем обученный scaler на трейн и тест
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    
    error_ = 0.5
    i = 0.00001
    while error_ >= 0.400:
        scores_2 = []
        model_2 = Lasso(alpha = i) 
        model_2.fit(X_train_scaled, Y_train)
        predictions_2 = model_2.predict(X_test_scaled)
        scores_2.append(metrics.mean_squared_error(predictions_2, Y_test))
        error_ = np.mean(scores_2)
        i = i + 0.00001
        print(error_, i)

0.4274373335623057 2e-05
0.42344358951549177 3.0000000000000004e-05
0.41541004958490724 4e-05
0.40855493213246014 5e-05
0.40287770029055925 6e-05
0.3983775155593553 7.000000000000001e-05
0.3875301242096766 2e-05
0.39542873129850875 2e-05
0.3835305547549985 2e-05
